# Assignment segment and cluster Boroughs of Toronto - part 1: getting table of Torontos neighbourhoods

## Matthias Reumann

In [1]:
# Install the lxml package in the current Jupyter kernel using conda
import sys
!conda install --yes --prefix {sys.prefix} lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h7b6447c_1         3.8 MB
    certifi-2019.6.16          |           py36_1         156 KB
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1     conda-forge --> 2019.6.16-py36_1    
    lxml:    4.2.5-py37hefd8a0e_0             --> 4.3.0-py36hefd8a0e_0
    openssl: 1.1.1

There seem to be issues in installing the lxml package using conda in this environment. 
Edit: The issue seems to be resolved and the results are shown. When running the code locally in a python script, the code executed perfectly.

## Scrap the wikipedia page

In [2]:
import pandas as pd


url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tables = pd.read_html(url, header=0)
print(tables[0].columns.values)

df = tables[0]
dfToronto = tables[0]
print(dfToronto.head(10))
print(dfToronto.shape)




['Postcode' 'Borough' 'Neighbourhood']
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned
(288, 3)


The table of Boroughs and neighbourhoods has 288 rows.

## Drop 'not assigned' fields in 'Borough'


In [3]:
# drop 'not assigned' fields in 'Borough'
# Get names of indexes
index = dfToronto[dfToronto['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
dfToronto.drop(index, inplace=True)
print(dfToronto.shape)
dfToronto.head(10)


(211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


After dropping the 'not assigned' rows in the Borough column, the table has 211 rows.

## Assign Neighbourhood 'Not assigned' to Borough name

In [4]:
# assign Neighbourhood 'Not assigned' to Borough name

index = dfToronto[dfToronto['Neighbourhood'] == 'Not assigned'].index

for i in index:
    print(i)
    dfToronto.loc[i]['Neighbourhood'] = dfToronto.loc[i]['Borough']

print(dfToronto.head(10))
print(dfToronto.shape)


8
   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Queen's Park
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern
(211, 3)


## Groupby Boroughs keeping the Neighbourhood as lists


In [7]:
# groupby Boroughs keeping the Neighbourhood as lists
dfToronto = dfToronto.groupby(['Borough']).agg({'Postcode': list, 'Neighbourhood': list}).reset_index()
print(dfToronto.columns.values)
dfToronto = dfToronto[['Postcode', 'Borough', 'Neighbourhood']]
print(dfToronto.shape)
dfToronto

['Borough' 'Postcode' 'Neighbourhood']
(11, 3)


,Postcode,Borough,Neighbourhood
0,"[[[M4N, M5N, M4P, M5P, M5P, M4R, M5R, M5R, M5R...",Central Toronto,"[[[Lawrence Park, Roselawn, Davisville North, ..."
1,"[[[M5A, M5A, M5B, M5B, M5C, M5E, M5G, M6G, M5H...",Downtown Toronto,"[[[Harbourfront, Regent Park, Ryerson, Garden ..."
2,"[[[M4E, M4K, M4K, M4L, M4L, M4M, M7Y]]]",East Toronto,"[[[The Beaches, The Danforth West, Riverdale, ..."
3,"[[[M4B, M4B, M4C, M4G, M4H, M4J]]]",East York,"[[[Woodbine Gardens, Parkview Hill, Woodbine H..."
4,"[[[M9A, M9B, M9B, M9B, M9B, M9B, M9C, M9C, M9C...",Etobicoke,"[[[Islington Avenue, Cloverdale, Islington, Ma..."
5,[[[M7R]]],Mississauga,[[[Canada Post Gateway Processing Centre]]]
6,"[[[M3A, M4A, M6A, M6A, M3B, M6B, M3C, M3C, M2H...",North York,"[[[Parkwoods, Victoria Village, Lawrence Heigh..."
7,[[[M7A]]],Queen's Park,[[[Queen's Park]]]
8,"[[[M1B, M1B, M1C, M1C, M1C, M1E, M1E, M1E, M1G...",Scarborough,"[[[Rouge, Malvern, Highland Creek, Rouge Hill,..."
9,"[[[M6H, M6H, M6J, M6J, M6K, M6K, M6K, M6P, M6P...",West Toronto,"[[[Dovercourt Village, Dufferin, Little Portug..."


Toronto seems to have 11 Boroughs after grouping them. The Neighbourhoods are assembled in a list. They could be converted further to a string using a join operation. 
Note: There is not a single post code per Borough. The post code seems to be associated with the Neighbourhoods. Thus, a good idea is to also aggregate them in a list, so that they are not lost. Having said that, I believe it makes more sense (see part 2 of the assignment) to group by Borough then Post code to aggregate the neighbourhoods.